# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ffa6c1521c0>
├── 'a' --> tensor([[-1.4886, -0.6883,  0.6741],
│                   [-0.3599, -0.5653, -0.3015]])
└── 'x' --> <FastTreeValue 0x7ffa6c152040>
    └── 'c' --> tensor([[-0.3019, -1.2061, -1.1235,  0.3280],
                        [ 0.9362,  0.9705, -0.1956, -0.7641],
                        [-0.8731,  0.4320, -0.1795,  0.9048]])

In [4]:
t.a

tensor([[-1.4886, -0.6883,  0.6741],
        [-0.3599, -0.5653, -0.3015]])

In [5]:
%timeit t.a

65 ns ± 0.0977 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ffa6c1521c0>
├── 'a' --> tensor([[ 0.0444, -2.5589, -2.3850],
│                   [ 1.2288, -1.3918,  1.3652]])
└── 'x' --> <FastTreeValue 0x7ffa6c152040>
    └── 'c' --> tensor([[-0.3019, -1.2061, -1.1235,  0.3280],
                        [ 0.9362,  0.9705, -0.1956, -0.7641],
                        [-0.8731,  0.4320, -0.1795,  0.9048]])

In [7]:
%timeit t.a = new_value

68.4 ns ± 0.0509 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.4886, -0.6883,  0.6741],
               [-0.3599, -0.5653, -0.3015]]),
    x: Batch(
           c: tensor([[-0.3019, -1.2061, -1.1235,  0.3280],
                      [ 0.9362,  0.9705, -0.1956, -0.7641],
                      [-0.8731,  0.4320, -0.1795,  0.9048]]),
       ),
)

In [10]:
b.a

tensor([[-1.4886, -0.6883,  0.6741],
        [-0.3599, -0.5653, -0.3015]])

In [11]:
%timeit b.a

57.8 ns ± 0.13 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.9081,  0.1128, -1.3189],
               [ 0.5357,  0.5668, -0.6491]]),
    x: Batch(
           c: tensor([[-0.3019, -1.2061, -1.1235,  0.3280],
                      [ 0.9362,  0.9705, -0.1956, -0.7641],
                      [-0.8731,  0.4320, -0.1795,  0.9048]]),
       ),
)

In [13]:
%timeit b.a = new_value

493 ns ± 0.199 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

828 ns ± 2.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 12.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

142 µs ± 922 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 261 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ffa6c156880>
├── 'a' --> tensor([[[-1.4886, -0.6883,  0.6741],
│                    [-0.3599, -0.5653, -0.3015]],
│           
│                   [[-1.4886, -0.6883,  0.6741],
│                    [-0.3599, -0.5653, -0.3015]],
│           
│                   [[-1.4886, -0.6883,  0.6741],
│                    [-0.3599, -0.5653, -0.3015]],
│           
│                   [[-1.4886, -0.6883,  0.6741],
│                    [-0.3599, -0.5653, -0.3015]],
│           
│                   [[-1.4886, -0.6883,  0.6741],
│                    [-0.3599, -0.5653, -0.3015]],
│           
│                   [[-1.4886, -0.6883,  0.6741],
│                    [-0.3599, -0.5653, -0.3015]],
│           
│                   [[-1.4886, -0.6883,  0.6741],
│                    [-0.3599, -0.5653, -0.3015]],
│           
│                   [[-1.4886, -0.6883,  0.6741],
│                    [-0.3599, -0.5653, -0.3015]]])
└── 'x' --> <FastTreeValue 0x7ffa6c156340>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 68.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff9c8a6ceb0>
├── 'a' --> tensor([[-1.4886, -0.6883,  0.6741],
│                   [-0.3599, -0.5653, -0.3015],
│                   [-1.4886, -0.6883,  0.6741],
│                   [-0.3599, -0.5653, -0.3015],
│                   [-1.4886, -0.6883,  0.6741],
│                   [-0.3599, -0.5653, -0.3015],
│                   [-1.4886, -0.6883,  0.6741],
│                   [-0.3599, -0.5653, -0.3015],
│                   [-1.4886, -0.6883,  0.6741],
│                   [-0.3599, -0.5653, -0.3015],
│                   [-1.4886, -0.6883,  0.6741],
│                   [-0.3599, -0.5653, -0.3015],
│                   [-1.4886, -0.6883,  0.6741],
│                   [-0.3599, -0.5653, -0.3015],
│                   [-1.4886, -0.6883,  0.6741],
│                   [-0.3599, -0.5653, -0.3015]])
└── 'x' --> <FastTreeValue 0x7ffa6c1b7c70>
    └── 'c' --> tensor([[-0.3019, -1.2061, -1.1235,  0.3280],
                        [ 0.9362,  0.9705, -0.1956, -0.7641],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 60.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.3 µs ± 66.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.4886, -0.6883,  0.6741],
                [-0.3599, -0.5653, -0.3015]],
       
               [[-1.4886, -0.6883,  0.6741],
                [-0.3599, -0.5653, -0.3015]],
       
               [[-1.4886, -0.6883,  0.6741],
                [-0.3599, -0.5653, -0.3015]],
       
               [[-1.4886, -0.6883,  0.6741],
                [-0.3599, -0.5653, -0.3015]],
       
               [[-1.4886, -0.6883,  0.6741],
                [-0.3599, -0.5653, -0.3015]],
       
               [[-1.4886, -0.6883,  0.6741],
                [-0.3599, -0.5653, -0.3015]],
       
               [[-1.4886, -0.6883,  0.6741],
                [-0.3599, -0.5653, -0.3015]],
       
               [[-1.4886, -0.6883,  0.6741],
                [-0.3599, -0.5653, -0.3015]]]),
    x: Batch(
           c: tensor([[[-0.3019, -1.2061, -1.1235,  0.3280],
                       [ 0.9362,  0.9705, -0.1956, -0.7641],
                       [-0.8731,  0.4320, -0.1795,  0.9048]],
         

In [26]:
%timeit Batch.stack(batches)

78.8 µs ± 263 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.4886, -0.6883,  0.6741],
               [-0.3599, -0.5653, -0.3015],
               [-1.4886, -0.6883,  0.6741],
               [-0.3599, -0.5653, -0.3015],
               [-1.4886, -0.6883,  0.6741],
               [-0.3599, -0.5653, -0.3015],
               [-1.4886, -0.6883,  0.6741],
               [-0.3599, -0.5653, -0.3015],
               [-1.4886, -0.6883,  0.6741],
               [-0.3599, -0.5653, -0.3015],
               [-1.4886, -0.6883,  0.6741],
               [-0.3599, -0.5653, -0.3015],
               [-1.4886, -0.6883,  0.6741],
               [-0.3599, -0.5653, -0.3015],
               [-1.4886, -0.6883,  0.6741],
               [-0.3599, -0.5653, -0.3015]]),
    x: Batch(
           c: tensor([[-0.3019, -1.2061, -1.1235,  0.3280],
                      [ 0.9362,  0.9705, -0.1956, -0.7641],
                      [-0.8731,  0.4320, -0.1795,  0.9048],
                      [-0.3019, -1.2061, -1.1235,  0.3280],
                      [ 0.9362,  

In [28]:
%timeit Batch.cat(batches)

141 µs ± 300 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

327 µs ± 1.92 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
